Test Phase4_3d file

In [25]:
# install anaconda: https://www.anaconda.com/products/individual
# if use Visual Studio Code, install python extension and select the python interpreter that includes anaconda
# install micropip: https://pypi.org/project/micropip/. You can also run anaconda prompt and type "pip install micropip"
import numpy as np
import micropip
await micropip.install("scipy")

In [26]:
from Phase4_trace_3d import Phase4_trace_3d
# from generateTestPositions import generateTestPositions
import generateTestPositions
from generateTestPositions import get_velocity_function
from generateTestPositions import generateTestPositions
from generateTestPositions import get_initial_position
# Cell 4
# Testing with dummy data
NumOfDataPoints = 1
clusterness = 0.1 # smaller number the more clustered
# Input conditions
initial_positions = []
v = []
# for i in range(1):
#     [x,y,z] = get_initial_position(i)
#     initial_positions.append([x, y, z])
#     vel = get_velocity_function(i)
#     v.append(vel)
initial_positions = np.zeros((NumOfDataPoints,3))
initial_positions[0] = get_initial_position(1)
v.append(lambda t: [np.sin(t), np.cos(t), 1])
noise = 1e-3
theta_degrees = 1.8
rev = 2  # revolutions of camera for the entire process
NOS = int(rev * 360 / theta_degrees)
NOS_per_section = 180  # must be larger than 5 to satisfy equations
camera_speed = 0.5  # in Hz or revolution per second
SRD = 1  # m, Source-Reference Distance
RDD = 1  # m, Reference-Detector (screen) Distance
radius = 1

method = 'acceleration'
dataPiling = 'overlap'

# Auto-calculations of the rest of the parameters derived from the setting above
delta_T = camera_speed * theta_degrees / 360
shots_per_second = 1 / delta_T

# Define the velocity function
# v = lambda t: [0.9 * np.sin(t), 0.9 * np.cos(t), 1]

# Pack conditions into a list
conditions = [noise, delta_T, NOS, theta_degrees, NOS_per_section, SRD, RDD,method,dataPiling]
print(conditions)

xz_proj = np.zeros((NOS, NumOfDataPoints*2))
real_positions = np.zeros((NOS, NumOfDataPoints*3))
# Generate test positions
for i in range(NumOfDataPoints):
    vel = v[i]
    xz_proj[:,i*2:i*2+2], real_positions[:,i*3:i*3+3]= generateTestPositions(vel, initial_positions[i], conditions)

[0.001, 0.0025, 400, 1.8, 180, 1, 1, 'acceleration', 'overlap']


In [27]:
# Print the first few rows of the generated data
print("xz_proj:")
print(xz_proj[:])
print("real_positions:")
print(real_positions[:])

xz_proj:
[[ 4.00637783e-01 -7.03727442e-04]
 [ 4.01651102e-01  5.83907382e-03]
 [ 4.03572855e-01  1.10618274e-02]
 [ 4.04754744e-01  1.48697191e-02]
 [ 4.05605517e-01  1.97249193e-02]
 [ 4.06288808e-01  2.43953512e-02]
 [ 4.07804172e-01  3.03118190e-02]
 [ 4.08286497e-01  3.47013072e-02]
 [ 4.10740979e-01  4.16912505e-02]
 [ 4.11056958e-01  4.70539823e-02]
 [ 4.12229587e-01  5.31319707e-02]
 [ 4.13501676e-01  5.71509849e-02]
 [ 4.13876818e-01  6.32143773e-02]
 [ 4.14673537e-01  6.87006180e-02]
 [ 4.14807767e-01  7.40853349e-02]
 [ 4.15304379e-01  7.97156016e-02]
 [ 4.16543442e-01  8.50578343e-02]
 [ 4.16272914e-01  9.04621507e-02]
 [ 4.17063026e-01  9.71825125e-02]
 [ 4.16027768e-01  1.03300232e-01]
 [ 4.16219368e-01  1.08351427e-01]
 [ 4.17004413e-01  1.14049552e-01]
 [ 4.16553107e-01  1.20965543e-01]
 [ 4.15374046e-01  1.26900323e-01]
 [ 4.16177027e-01  1.32134099e-01]
 [ 4.14700191e-01  1.38340975e-01]
 [ 4.14649504e-01  1.44464404e-01]
 [ 4.12989374e-01  1.51542865e-01]
 [ 4.121942

In [28]:
# Import the function directly from the module
from Phase4_trace_3d import Phase4_trace_3d
from smooth_points import smooth_points
from particle_path_finder import ParticlePathFinder as pf
from particle_path_finder import OriginalPathKeeper as opk
import pandas as pd
import random

# read data from .csv file
input_data = pd.read_csv('particle_tracking_input.csv')

columnIndex = 0

# Ai recognition caller
alpha = np.radians(theta_degrees)  # Example rotation angle in radians
path_finder = pf(alpha)
original_path_keeper = opk(alpha)
# iterate through each row of the input data as a snapshot, and put the snapshot into the particle path finder
for _, row in input_data.iterrows():
    
    values = row.tolist()  

    # input format, list of tuple of two elements (x,y)
    paired_values = [(values[i], values[i+1]) for i in range(0, len(values)-1, 2)]
    scambled_values = random.shuffle(paired_values.copy())
    path_finder.append(scambled_values)
    original_path_keeper.append(paired_values, 0)
    
    

    

estimated_positions = np.zeros((NOS,3*NumOfDataPoints))
# Then you can call it just by its name:
for i in range(NumOfDataPoints):
    estimated_positions_single = Phase4_trace_3d(conditions, xz_proj[:,i*2:i*2+2])
    estimated_positions_single = smooth_points(estimated_positions_single, 'sg',NOS_per_section)

    estimated_positions[:,i*3:i*3+3] = estimated_positions_single



NOS:  400
NOS_per_Section:  180


In [29]:
print('estimated positions: \n',estimated_positions)
# print('real positions: \n',real_positions)

estimated positions: 
 [[ 1.95172738e-01 -7.66258884e-03 -3.26470413e-04]
 [ 1.95391509e-01 -5.12760946e-03  2.18265227e-03]
 [ 1.95615109e-01 -2.59401802e-03  4.69171899e-03]
 ...
 [ 6.47182754e-01  7.76931155e-01  9.45203197e-01]
 [ 6.48623917e-01  7.77335659e-01  9.46625180e-01]
 [ 6.50062621e-01  7.77726867e-01  9.48036979e-01]]


In [30]:
import matplotlib
print(matplotlib.get_backend())
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')  # or another interactive backend

# from mpl_toolkits.mplot3d import Axes3D
import os


def Phase4Graph(real_positions, estimated_positions, conditions, v, f2):
    noise, delta_T, NOS, theta_degree, N, SRD, RDD, method, dataPiling = conditions
    
    plt.ion()
    # Ensure that real_positions and estimated_positions are the same length
    real_positions = real_positions[:len(estimated_positions)]
    
    fig = plt.figure(f2, figsize=(20, 16), dpi=100)
    
    ax1 = fig.add_subplot(111, projection='3d')
    
    for i in range(NumOfDataPoints):
        ax1.plot(real_positions[:, i*3], real_positions[:, i*3+1], real_positions[:, i*3+2], 'r', markersize=10)
        ax1.plot(estimated_positions[:, i*3], estimated_positions[:, i*3+1], estimated_positions[:, i*3+2], 'b', markersize=20)
    
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_xlim([-0.5, 0.5])
    ax1.set_ylim([-0.5, 0.5])
    ax1.set_zlim([0, 1.5])
    ax1.set_title('Real Positions vs Estimated Positions in 3D')
    ax1.legend(['Real Positions', 'Estimated Positions'])
    
    distances = np.linalg.norm(real_positions - estimated_positions, axis=1)
    min_val, min_idx = np.min(distances), np.argmin(distances)
    max_val, max_idx = np.max(distances), np.argmax(distances)
    
    # ax1.text(*real_positions[min_idx], f'Min ({min_idx}): {real_positions[min_idx]}', color='r')
    # ax1.text(*estimated_positions[min_idx], f'Min ({min_idx}): {estimated_positions[min_idx]}', color='b')
    # ax1.text(*real_positions[max_idx], f'Max ({max_idx}): {real_positions[max_idx]}', color='r')
    # ax1.text(*estimated_positions[max_idx], f'Max ({max_idx}): {estimated_positions[max_idx]}', color='b')
    
    print(f'Noise: {noise}\nMin Distance: {min_val}\nMax Distance: {max_val}\nNumber of Shots: {NOS}\nRotation: {theta_degree}\nTime Between Shots: {delta_T}\nNOS Per Section: {N}\nMethod: {method}\nData Piling: {dataPiling}')
    
    v_str = str(v).replace('<function ', '').replace(' at 0x' + hex(id(v))[2:] + '>', '')
    
    print(f'True velocity:\n{v_str}\nt is time elapsed')
    
    
    plt.show()


TkAgg


In [31]:
f2 = 0
Phase4Graph(real_positions, estimated_positions, conditions, v, f2)

Noise: 0.001
Min Distance: 0.007166059411855679
Max Distance: 0.07997463023347912
Number of Shots: 400
Rotation: 1.8
Time Between Shots: 0.0025
NOS Per Section: 180
Method: acceleration
Data Piling: overlap
True velocity:
[<lambda> at 0x000001EDBD33D940>]
t is time elapsed
